In [1]:
rdd = sc.textFile("../data/taxi-data-sorted-small.csv.bz2")
rdd = rdd.map(lambda x: x.split(','))

In [2]:
def isfloat(value):
    try:
        float(value)
        return True
    except:
        return False

def correctRows(p):
    if(len(p) == 17):
        if(isfloat(p[5]) and isfloat(p[11])):
            if(float(p[5]) != 0 and float(p[11]) != 0):
                return p

In [3]:
texilinesCorrect = rdd.filter(correctRows)

In [4]:
n = texilinesCorrect.count()

In [5]:
distance_money = texilinesCorrect.map(lambda x: (1, (float(x[5]), float(x[11]))))

In [37]:
distance_money.take(1)

[(1, (0.44, 3.5))]

In [38]:
def getSumPart(x):
    partOfDm = x[1][0] * (x[1][1] - (m * x[1][0] + b))
    partOfDb = (x[1][1] - (m * x[1][0] + b))
    return (x[0], (partOfDm, partOfDb))

In [39]:
sumInGradient = distance_money.map(getSumPart)\
                            .reduceByKey(lambda x1, x2: (x1[0]+x2[0], x1[1]+x2[1]))

In [40]:
sumInGradient.collect()

[(1, (-9.579758625477552e-06, -2.767919795587659e-07))]

In [52]:
def gradientDescent(learningRate = 0.01, num_iteration = 100):
    prev_m = 0.1
    prev_b = 0.1
    count = 0
    precision = 0.00000001
    prev_stepsize = 1
    while(prev_stepsize > precision or num_iteration > count):
        
        
        def getSumPart(x):
            partOfDm = - x[1][0] * (x[1][1] - (prev_m * x[1][0] + prev_b))
            partOfDb = - (x[1][1] - (prev_m * x[1][0] + prev_b))
            return (x[0], (partOfDm, partOfDb))
        
        sumInGradient = distance_money.map(getSumPart)\
                            .reduceByKey(lambda x1, x2: (x1[0]+x2[0], x1[1]+x2[1]))\
                            .collect()
        
        dm = (2/n) * sumInGradient[0][1][0]
        db = (2/n) * sumInGradient[0][1][1]
        
        cur_m = prev_m - (learningRate * dm)
        cur_b = prev_b - (learningRate * db)
        
        print("Iteration: ", count)
        print("m: ", cur_m, "b: ", cur_b)
        
        count += 1
        
        prev_stepsize = mean([abs(prev_m - cur_m), abs(prev_b - cur_b)])
        
        prev_m = cur_m
        prev_b = cur_b
        
        
    return cur_m, cur_b

In [53]:
m, b = gradientDescent()

Iteration:  0
m:  1.3783430859133066 b:  0.3283392434996293
Iteration:  1
m:  2.12515132707956 b:  0.4778625395155822
Iteration:  2
m:  2.5605001030744776 b:  0.5810189965931698
Iteration:  3
m:  2.8133559916522177 b:  0.6568262472478248
Iteration:  4
m:  2.959294694934295 b:  0.7164308902365072
Iteration:  5
m:  3.042604208292947 b:  0.7663669784272836
Iteration:  6
m:  3.08923754928993 b:  0.8104655330573136
Iteration:  7
m:  3.1144032161990456 b:  0.8509735428503082
Iteration:  8
m:  3.1270132900739225 b:  0.8892097115299928
Iteration:  9
m:  3.1322900772625295 b:  0.9259487341899088


KeyboardInterrupt: 